In [2]:
import os
from lxml import etree
import re
import csv

In [8]:
# set dtsx package file
xml_file = rf'./dtsx_files/SSIS_SQLExtractSample.dtsx'#HR_DataConductor.dtsx'
if not os.path.isfile(xml_file):
    print("no package file")

In [10]:
# read and parse ssis package
tree = etree.parse(xml_file)
root = tree.getroot()
root

<Element {www.microsoft.com/SqlServer/Dts}Executable at 0x1e9eb08fbc0>

In [11]:
# Define the tags we needs to search the SSIS package for (all unique tags available in ele_tags variable)
pfx = '{www.microsoft.com/'
ref_tag = pfx + 'SqlServer/Dts}refId'
exe_tag = pfx + 'SqlServer/Dts}Executable'
obj_tag = pfx + 'SqlServer/Dts}ObjectName'
dat_tag = pfx + 'SqlServer/Dts}ObjectData'
tsk_tag = pfx + 'sqlserver/dts/tasks/sqltask}SqlTaskData'
src_tag = pfx + 'sqlserver/dts/tasks/sqltask}SqlStatementSource'
desc_tag = pfx + 'SqlServer/Dts}Description'

In [12]:
# Define the namespace
namespaces = {
              'DTS': 'www.microsoft.com/SqlServer/Dts',
              'SQLTask': 'www.microsoft.com/sqlserver/dts/tasks/sqltask'
             }

In [13]:
# Loop over the found elements and extract the text between the property tags
def extract_data_flow(id, element):
        for comp in element.xpath('.//component', namespaces=namespaces):
            task_name = comp.get('name')
            ssis_path = comp.get('refId')

            for prop in comp.xpath('.//property', namespaces=namespaces):
                # If prop.text is not null
                if prop.text and prop.get('name') == 'SqlCommand':
                    with open(f'./sql_scripts/{id}-{task_name}.sql', "w") as file: 
                        file.write(f'-- SSIS Location: {ssis_path}\n')
                        file.write(prop.text.strip())

In [14]:
def extract_sql_task(id, element, ssis_path, file_name):
    for obj in element.xpath('.//SQLTask:SqlTaskData', namespaces=namespaces):
        sql_query = obj.get(src_tag)
        if sql_query:
            with open(f'./sql_scripts/{id}-{file_name}.sql', "w") as file: 
                file.write(f'-- SSIS Location: {ssis_path}\n')
                file.write(sql_query.strip())
    

In [15]:
package_name = root.get(obj_tag)

for id, execute in enumerate(root.xpath('//DTS:Executable', namespaces=namespaces)):
    refid = execute.get(ref_tag)
    obj_name = execute.get(obj_tag)
    
    if execute.get(desc_tag) == 'Data Flow Task':
        extract_data_flow(id=id, element=execute)

    elif execute.get(desc_tag) == 'Execute SQL Task':
        extract_sql_task(id=id, element=execute, ssis_path=refid, file_name=obj_name)
        
        

In [59]:
task_namespace = {'SQLTask': 'www.microsoft.com/sqlserver/dts/tasks/sqltask'}

In [68]:
for id, execute in enumerate(root.xpath('.//SQLTask:SqlTaskData', namespaces=namespaces)):
    print(execute.get(src_tag))

        #'{www.microsoft.com/SqlServer/Dts}Executable'

SELECT * FROM Test.SQLTask;
